In [1]:
# Import PyTorch
import torch
import wandb
from torch import nn
import torch.nn as nn 
import numpy as np
import torchvision.transforms as transforms 
import torch.optim as optim
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.models import resnet50, ResNet50_Weights
import torchvision.models as models



# Import matplotlib for visualization
import matplotlib.pyplot as plt


In [2]:
num_epochs = 20
batch_size = 64
num_classes = 211
initial_lr = 0.001

run = wandb.init(
    project="country-classificaton",
    notes="",
    tags=["baseline", "paper1"]
)

wandb.config = {
  "epochs": 20, 
  "learning_rate": 0.001, 
  "batch_size": 64
}

transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=30),
    transforms.RandomPosterize(bits=4),
    transforms.RandomSolarize(threshold=128),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.ToTensor(), # convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.ToTensor(), # convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chloechiaw. Use `wandb login --relogin` to force relogin


In [3]:
# use virtual machine to download the data as its faster 
trainset = torchvision.datasets.Country211(root='./', split='train', transform=transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2) # for trainloader, set shuffle to true 

valset = torchvision.datasets.Country211(
    root='./', split='valid', transform=transform)
valloader = torch.utils.data.DataLoader(
    valset, batch_size=64, shuffle=False, num_workers=2) # for testloader, set shuffle to false 

testset = torchvision.datasets.Country211(
    root='./', split='test', transform=test_transform)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=False, num_workers=2) # for testloader, set shuffle to false 

In [9]:
resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)
num_features = resnet.fc.in_features #in_features is an attribute, 2048 features 
resnet.fc = nn.Identity()  # Remove the final fully connected layer
custom_fc = nn.Sequential(
    nn.Linear(num_features, num_classes),
    nn.Softmax(dim=1),
    # nn.Dropout(0.25),  # Dropout layer with a dropout rate of 0.0.25
)
model = nn.Sequential(resnet, custom_fc)
model.to(device=0)


Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.3333333333, end_factor = 1.0, total_iters=30)
scheduler.last_epoch = -1

In [11]:
torch.cuda.is_available()
device = 0

In [12]:
num_classes = 211 # number of classes in country211

In [13]:
n_epochs = 20
print_every = 10
valid_loss_min = np.Inf # infinity 
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(trainloader)
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, target_) in enumerate(trainloader):
        data_, target_ = data_.to(device), target_.to(device) # target is a One Hot Encoding with 210 zeroes and one 1 to represent the actual country class that is represented 
        optimizer.zero_grad()
        
        outputs = resnet(data_)
        loss = criterion(outputs, target_) # 
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        wandb.log({"train-loss": f"{np.mean(train_loss):.4f}", "train-acc": f"{100 * correct / total:.4f}"})
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
    batch_loss = 0
    total_t=0

    correct_t=0
    with torch.no_grad():
        resnet.eval()
        for data_t, target_t in (valloader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = resnet(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
            wandb.log({"val-loss": f"{np.mean(val_loss):.4f}", "val-acc": f"{(100 * correct_t/total_t):.4f}"})
        val_acc.append(100 * correct_t/total_t)
        val_loss.append(batch_loss/len(valloader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t/total_t):.4f}\n')

        if network_learned:
            valid_loss_min = batch_loss
            torch.save(resnet.state_dict(), 'resnetmonday1.pt')
            wandb.save("modely")
            print('Improvement-Detected, save-model')
    resnet.train()

Epoch 1



/home/chloecookie/miniconda3/envs/country/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/chloecookie/miniconda3/envs/country/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch [1/20], Step [0/495], Loss: 7.7123
Epoch [1/20], Step [20/495], Loss: 7.6313


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
testmodel = torch.load('resnet2.pt')
model.eval() 
correct = 0 
total = 0

with torch.no_grad():
    for (images, labels) in iter(testloader): # make sure to use next(iter) 
        images, labels = images.cuda(), labels.cuda() # need to add this line to make sure images and labels are on the same device 
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) # # .max() applies a softmax to the output. torch.max(outputs.data, 1) returns a tuple containing two elements. The first element represents the maximum values along the specified dimension (in this case, dimension 1), and the second element represents the indices of those maximum values.

# By using _ as a placeholder, we are indicating that we don't need or care about the first element (maximum values), and we are only interested in the second element (indices of maximum values). This is a way to discard or ignore the unwanted value.
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy of the model on the test images: {:.2f}%'.format(accuracy))

Accuracy of the model on the test images: 0.53%


In [14]:
images, labels in iter(testloader)
images, labels = images.cuda(), labels.cuda() # need to add this line to make sure images and labels are on the same device 
outputs = model(images) # this outputs the prediction for each image 
outputs.data


tensor([[-0.0840, -0.0248,  0.0505,  ...,  0.2406, -0.1043, -0.0738],
        [-0.0249, -0.1131,  0.0785,  ..., -0.0610,  0.0468, -0.0250],
        [-0.0082, -0.0154,  0.2432,  ..., -0.1125, -0.0475,  0.0267],
        ...,
        [-0.1111, -0.0262, -0.1376,  ...,  0.1256, -0.0388, -0.0499],
        [-0.0926, -0.0881,  0.0309,  ..., -0.0138, -0.0013, -0.0086],
        [-0.0402,  0.0543,  0.0291,  ..., -0.0584, -0.0365,  0.1925]],
       device='cuda:0')

AttributeError: 'collections.OrderedDict' object has no attribute 'cuda'